## Tinkoff Data Science Challenge (восстановление позиции мерчанта)
По правилам соревнования разрешено использовать внешние (открытые и бесплатные) данные. 

Полезно будет знать для каждой точки её адрес (город, улица, номер дома). Это позволит, например, проверять такие идеи:
- для каждого мерчанта искать город, в котором большинство точек, остальные фильтровать как плохие;
- если точка находится в здании (у нее есть номер дома), то она более вероятно близка к мерчанту;
- и.т.д.

Также нужны данные по организациям в каждом доме:
- чем больше организаций в доме, тем вероятнее, что там мерчант,
- если в трейн данных много мерчантов в домах, где есть какие-то сетевые магазины, то более вероятно, что в тестовых данных мерчанты будут в домах с такими же магазинами;
- и.т.д.

In [1]:
import os
import json
import requests
import time
import random
from collections import defaultdict, Counter
import math

import pandas as pd
import numpy as np

import folium
from folium.plugins import HeatMap

from ipywidgets import widgets
from IPython.display import display, clear_output

from tqdm import tqdm

import osmium

import matplotlib.pyplot as plt
%matplotlib inline  

pd.options.display.max_rows = 100

RS = 113113

OSM_URL = 'http://localhost:8080/reverse?format=json&lat={}&lon={}' + \
'&zoom=18&addressdetails=1&accept-language=ru_RU'
OSM_DIR = 'osm_geocoder'

if not os.path.exists(OSM_DIR):
    os.makedirs(OSM_DIR)
    
HSIDE = 0.002

In [2]:
COLOR_TRANS = '#cc8631'
COLOR_MERCHANT = 'blue'
COLOR_LINE = '#009955'
COLOR_BUILDING = '#000000'
COLOR_SHOP = '#ff00ff'

В данных есть точки с нечисловыми координатами, отфильтруем их и заменим разделитель на запятую.
Также заменим названия полей на mid, lat, lon, ttime, rectime.

In [8]:
transactions = pd.read_csv('data/transactions.csv', sep=';', encoding='cp1251')
lon_null = pd.to_numeric(transactions.longitude, errors='coerce').isnull()
lat_null = pd.to_numeric(transactions.latitude, errors='coerce').isnull()
transactions.columns = ['mid', 'lat', 'lon', 'ttime', 'rectime']
transactions[~lat_null & ~lon_null].to_csv('data/transactions_filter_bad.csv', index=False)

train_merchants = pd.read_csv('data/merchants_train.csv', sep=';')
train_merchants.columns = ['mid', 'lat', 'lon']
train_merchants.to_csv('data/mtrain.csv', index=False)

test_merchants = pd.read_csv('data/merchants_test.csv', sep=';')
test_merchants.columns = ['mid', 'lat', 'lon']
test_merchants.to_csv('data/mtest.csv', index=False)

### Геокодер
Узнать адрес по широте и долготе (reverse geocoding) можно с помощью сервисов геокодера. 
Хорошее качество для России у геокодера Яндекса (но он бесплатный только для 25к запросов в сутки). 
Можно воспользовать геокодером Nominatim, который работает с данными OpenStreetMap.
Проще всего поднять Nominatim локально из докер образа, например https://github.com/mediagis/nominatim-docker.
Для этого потребуется пересобрать образ с картой России (может занять до 10ти часов).

Nominatim даже локально работает довольно медленно, закэшируем данные в файлы.

In [3]:
def float_to_str(f):
    return '{:.10f}'.format(f).rstrip('0').rstrip('.')

def get_filename(lat, lon):
    return '{}/{},{}'.format(OSM_DIR, float_to_str(lat), float_to_str(lon))

In [12]:
def download(data):
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        lat = row.lat
        lon = row.lon
        filename = get_filename(lat, lon)
        if os.path.isfile(filename):
            continue
        response = requests.get(OSM_URL.format(float_to_str(lat), float_to_str(lon)))
        if response.status_code != 200:
            print response.status_code, response.url
            break
        with open(filename, 'w') as fout:
            fout.write(response.text.encode('utf-8'))

# скачиваем для точек мерчантов
download(pd.read_csv('data/mtrain.csv'))
# скачиваем для точек чекинов
download(pd.read_csv('data/transactions_filter_bad.csv'))

100%|██████████| 249283/249283 [00:25<00:00, 9785.52it/s] 


Функции получения страны, города, номера дома по координатам. Если не может определить, то возвращает пустую строку.

In [4]:
def get_osm_country(lat, lon):
    filename = get_filename(lat, lon)
    with open(filename) as f:
        json_data = json.load(f)
    if 'address' not in json_data:
        return ''
    if 'country' not in json_data['address']:
        return ''
    country = json_data['address']['country'].encode('utf-8')
    if country == 'Россия' or country == 'РФ':
        return 'РФ'
    if country == 'Украина':
        if 'state' in json_data['address']:
            state = json_data['address']['state'].encode('utf-8')
            if state == 'Автономная Республика Крым' or state == 'Республика Крым' or state == 'Севастополь':
                return 'РФ'
    return country

def get_osm_city(lat, lon):
    filename = get_filename(lat, lon)
    with open(filename) as f:
        json_data = json.load(f)
    if 'address' not in json_data:
        return ''
    if 'state' in json_data['address']:
        state = json_data['address']['state'].encode('utf-8')
        # в OSM Москва, Спб, Севастополь указываются на уровне state
        if state == 'Москва' or state == 'Санкт-Петербург' or state == 'Севастополь':
            return state
    if 'city' in json_data['address']:
        return json_data['address']['city'].encode('utf-8')
    if 'town' in json_data['address']:
        return json_data['address']['town'].encode('utf-8')
    if 'village' in json_data['address']:
        return json_data['address']['village'].encode('utf-8')
    if 'hamlet' in json_data['address']:
        return json_data['address']['hamlet'].encode('utf-8')
    if 'county' in json_data['address']:
        # для Рязани почему-то не определяется город, но можно определить по другим полям
        county = json_data['address']['county'].encode('utf-8')
        if county == 'городской округ Рязань' and 'city_district' in json_data['address']:
            return 'Рязань'
    # эти точки определяется без города и в городском округе Балашиха
    if abs(lat - 55.812529) < 1e-10 and abs(lon - 37.832827) < 1e-10:
        return 'Москва'
    if abs(lat - 55.81257) < 1e-10 and abs(lon - 37.833357) < 1e-10:
        return 'Москва'
    # точка на ЕКАД, входит в Екатеринбург
    if abs(lat - 56.774367) < 1e-10 and abs(lon - 60.455298) < 1e-10:
        return 'Екатеринбург'
    return ''

def get_osm_number(lat, lon):
    filename = get_filename(lat, lon)
    with open(filename) as f:
        json_data = json.load(f)
    if 'address' not in json_data:
        return ''
    if 'house_number' in json_data['address']:
        return json_data['address']['house_number'].encode('utf-8')
    return ''

Посмотрим на распределение самых частых мерчантов по городам. Для некоторых не определяется город 
(хотя на самом деле мерчант находится в городе) - подправим функцию get_osm_city.

In [14]:
cities_dict = defaultdict(int)
train_merchants = pd.read_csv('data/mtrain.csv')
train_count = train_merchants.shape[0]
for _, row in train_merchants.iterrows():
    lat, lon = row.lat, row.lon
    if get_osm_country(lat, lon) == 'РФ':
        cities_dict[get_osm_city(lat, lon)] += 1

print 'Не в городе: {}\n'.format(cities_dict[''])
cities = sorted([(k, v) for k, v in cities_dict.iteritems()], reverse=True, key=lambda x: x[1])
for city, count in cities[:30]:
    print u'{:>35} {:>4} ({:.3f})'.format(city.decode('utf-8'), count, count * 1.0 / train_count)

Не в городе: 14

                             Москва 3214 (0.496)
                    Санкт-Петербург 1163 (0.179)
                       Екатеринбург  158 (0.024)
                             Казань  132 (0.020)
                        Новосибирск  106 (0.016)
                              Химки   60 (0.009)
                          Краснодар   51 (0.008)
                              Пермь   50 (0.008)
                     Ростов-на-Дону   48 (0.007)
                    Нижний Новгород   46 (0.007)
                             Самара   45 (0.007)
                        Калининград   41 (0.006)
                          Челябинск   33 (0.005)
                         Котельники   32 (0.005)
                            Воронеж   32 (0.005)
                             Мытищи   30 (0.005)
                               Сочи   28 (0.004)
                                Уфа   28 (0.004)
                     Усть-Щербедино   28 (0.004)
                           Балашиха   28 (0.004)
   

In [5]:
# Показать мерчанты на карте
# merchants - список из (mid, lat, lon)
# transactions - dataframe транзакций, если None, то не показывать транзакции для каждого мерчанта
# print_info - выводить ли mid и координаты
# show_lines - рисовать ли линии от мерчанта к точкам транзакций
def show_merchants(merchants, transactions=None, print_info=False, show_lines=True):
    count_merchants = len(merchants)
    
    def show_current_merchant(current_merchant):
        clear_output()
        status_text.value = '{} from {}'.format(current_merchant + 1, count_merchants)

        mlat, mlon, mid = merchants[current_merchant]
        if print_info:
            print '{},{},{}'.format(mid, mlat, mlon)
        folium_map = folium.Map(location=[mlat, mlon], zoom_start=16)
        folium.Marker([mlat, mlon]).add_to(folium_map)
        folium.features.RectangleMarker(
                bounds=[[mlat - HSIDE, mlon - HSIDE], 
                        [mlat + HSIDE, mlon + HSIDE]],
                color=COLOR_MERCHANT, fill_color=COLOR_MERCHANT, fill_opacity=0.3).add_to(folium_map)

        if transactions is not None:
            mid_trans = transactions[transactions.mid == mid]
            if print_info:
                print 'Transactions: {}'.format(mid_trans.shape[0]) 
            for _, row in mid_trans.iterrows():
                lat = row.lat
                lon = row.lon
                folium.CircleMarker(location=[lat, lon], radius=8, 
                                    color=COLOR_TRANS, fill_color=COLOR_TRANS).add_to(folium_map)
                if show_lines:
                    folium.PolyLine(locations=[[lat, lon], [mlat, mlon]], 
                                    color=COLOR_LINE, weight=1, opacity=1).add_to(folium_map)
        display(folium_map)
        
    current_merchant = [0]

    def show_next_merchant(_):
        current_merchant[0] = (current_merchant[0] + 1) % count_merchants
        show_current_merchant(current_merchant[0])

    def show_prev_merchant(_):
        current_merchant[0] = (current_merchant[0] - 1) % count_merchants
        show_current_merchant(current_merchant[0])

    container = widgets.HBox()
    button_next = widgets.Button(description='Next')
    button_prev = widgets.Button(description='Prev')
    status_text = widgets.Text()
    container.children = [button_prev, status_text, button_next]
    display(container)
    button_next.on_click(show_next_merchant)
    button_prev.on_click(show_prev_merchant)
    show_current_merchant(current_merchant[0])

Посмотрим на точки не в городе, если это ошибка OSM, то поправим функцию get_osm_city.

In [8]:
train_merchants = pd.read_csv('data/mtrain.csv')
mids = []
for _, row in train_merchants.iterrows():
    lat, lon = row.lat, row.lon
    if get_osm_country(lat, lon) == 'РФ' and get_osm_city(lat, lon) == '':
        mids.append((lat, lon, int(row.mid)))
        
show_merchants(mids, print_info=True)

403171,55.414327,37.90047


Среди точек транзакций очень много плохих частых точек.

Посмотрим на тепловую карту мерчантов для самых частых точек.

In [9]:
train_merchants = pd.read_csv('data/mtrain.csv')
transactions = pd.read_csv('data/transactions_filter_bad.csv')

transactions_count = transactions.groupby(['lat', 'lon']).size()
transactions_count = transactions_count[transactions_count > 80]

latlons = [(ll[0], ll[1], count) for ll, count in transactions_count.iteritems()]
latlons = sorted(latlons, key=lambda x: x[2], reverse=True)

count_points = len(latlons)
current_point = 0

def show_current_point(current_point):
    clear_output()
    status_text.value = '{}/{}'.format(current_point + 1, count_points)

    tlat, tlon, tcount = latlons[current_point]
    all_mids = set(transactions[(transactions.lat == tlat) & 
                   (transactions.lon == tlon)]['mid'].unique())
    train_mids = train_merchants[train_merchants.mid.apply(lambda x: x in all_mids)]
    train_ll = zip(train_mids.lat.values, train_mids.lon.values)
    train_size = len(train_ll)
    
    folium_map = folium.Map(location=[tlat, tlon], zoom_start=13)
    folium.Marker([tlat, tlon]).add_to(folium_map)
    folium.features.RectangleMarker(
            bounds=[[tlat - HSIDE, tlon - HSIDE], 
                    [tlat + HSIDE, tlon + HSIDE]],
            color=COLOR_MERCHANT, fill_color=COLOR_MERCHANT, fill_opacity=0.3).add_to(folium_map)
    HeatMap(train_ll).add_to(folium_map)

    correct = sum(abs(mlat - tlat) < HSIDE and abs(mlon - tlon) < HSIDE for mlat, mlon in train_ll)

    correct = 0 if train_size == 0 else correct * 1.0 / train_size
    print '             {}, {}\n'.format(tlat, tlon)
    print '                    correct: {:.3f}'.format(correct)
    print 'count transactions in point: {}'.format(tcount)
    print '      count train merchants: {}'.format(train_size)
    print '       count test merchants: {}'.format(len(all_mids) - train_size)
    print 
    
    cities = defaultdict(int)
    for mlat, mlon in train_ll:
        cities[get_osm_city(mlat, mlon)] += 1
    cities = sorted([(city, count) for city, count in cities.iteritems()], reverse=True, key=lambda x: x[1])
    sum_cities = sum([x[1] for x in cities])
    
    if len(cities) > 5:
        cities = cities[:5]
    cities = [(city, count * 1.0 / sum_cities) for city, count in cities]
    for city, count in cities:
        print u'{:>27}: {:.3f}'.format(city.decode('utf-8'), count)

    display(folium_map)
    
def show_next_point(_):
    global current_point
    current_point = (current_point + 1) % count_points
    show_current_point(current_point)
    
def show_prev_point(_):
    global current_point
    current_point = (current_point - 1) % count_points
    show_current_point(current_point)

container = widgets.HBox()
button_next = widgets.Button(description='Next')
button_prev = widgets.Button(description='Prev')
status_text = widgets.Text(value='No map')
container.children = [button_prev, status_text, button_next]
display(container)
button_next.on_click(show_next_point)
button_prev.on_click(show_prev_point)
show_current_point(current_point)

             55.75034704, 37.62385111

                    correct: 0.000
count transactions in point: 27955
      count train merchants: 3440
       count test merchants: 1881

                     Москва: 0.507
            Санкт-Петербург: 0.189
               Екатеринбург: 0.024
                     Казань: 0.022
                Новосибирск: 0.015


Будем считать точку плохой, если:
она не попадает ни в один прямоугольник мерчанта
не сильно смещает частоту мерчантов по городам относительно частоты для всех мерчантов
на тепловой карте не видно кластеров.

Плохие точки: 

55.75034704, 37.62385111 (27955)

55.75034704, 37.6235111 (722)

55.750347, 37.623851 (471)

55.750347043, 37.623851113 (150)

0.0, 0.0 (25018)

51.17889992, -1.82639994 (5420)

51.1789, -1.8264 (228)

51.172, -1.2634 (177)

Отфильтруем эти точки.

In [9]:
transactions = pd.read_csv('data/transactions_filter_bad.csv')
print transactions.shape

BAD_POINTS = [(55.75034704, 37.62385111), (55.75034704, 37.6235111), 
              (55.750347, 37.623851), (55.750347043, 37.623851113), 
              (0, 0), (51.17889992, -1.82639994),
              (51.1789, -1.8264), (51.172, -1.2634)]
for lat, lon in BAD_POINTS:
    transactions = transactions[(abs(transactions.lat - lat) > 1e-10) | 
                                (abs(transactions.lon - lon) > 1e-10)]

print transactions.shape
transactions.to_csv('data/transactions_filter_bad_freq.csv', index=False)

(249283, 5)
(189142, 5)


Возможно у некоторых мерчантов все точки были плохими. Посмотрим, есть ли среди них мерчанты с повторяющимися координатами.

In [10]:
transactions = pd.read_csv('data/transactions_filter_bad_freq.csv')
train_merchants = pd.read_csv('data/mtrain.csv')
train_mids = set(train_merchants.mid)
trans_mids = set(transactions.mid)

bad_mids = train_mids - trans_mids
print 'only bad:', len(bad_mids)
train_merchants_bad = train_merchants[train_merchants.mid.apply(lambda x: x in bad_mids)]
train_merchants_bad_size = train_merchants_bad.groupby(['lat', 'lon']).size()
print train_merchants_bad_size[train_merchants_bad_size > 1]
bad_ll = zip(train_merchants_bad.lat.values, train_merchants_bad.lon.values)
folium_map = folium.Map(location=[55, 37], zoom_start=3)
HeatMap(bad_ll).add_to(folium_map)
folium_map

only bad: 96
lat        lon      
55.740213  37.656371    4
55.842454  37.565317    2
dtype: int64


На карте никаких кластеров нет, самая частая точка мерчанта в трейне из тех, у кого только 
плохие точки транзакций, - 55.740213 37.656371. Для мерчантов из теста, у кого только плохие 
транзакции стоит предсказывать её.

Кроме откровенно плохих точек есть еще точки транзакций, которые находятся не в России и скорее всего тоже шумные.
Проверим сколько мерчантов находится не в России.

In [8]:
train_merchants = pd.read_csv('data/mtrain.csv')
transactions = pd.read_csv('data/transactions_filter_bad_freq.csv')

not_russia = []
for index, row in train_merchants.iterrows():
    lat = row.lat
    lon = row.lon
    country = get_osm_country(lat, lon)
    if country != 'РФ':
        not_russia.append((lat, lon, row.mid))

show_merchants(not_russia, transactions)

Таких мерчантов всего 5, причем у 4ех все точки в России.
Предсказать такие мерчанты невозможно, поэтому надо удалить все точки транзакций не в Росиии.

In [11]:
transactions = pd.read_csv('data/transactions_filter_bad_freq.csv')
transactions = transactions[[get_osm_country(row.lat, row.lon) == 'РФ' for _, row in transactions.iterrows()]]
transactions.to_csv('data/transactions_good.csv', index=False)

Посмотрим на мерчанты, которые находятся в тех же точках.

In [14]:
train_merchants = pd.read_csv('data/mtrain.csv')
transactions = pd.read_csv('data/transactions_good.csv')

size = train_merchants.groupby(['lat', 'lon']).size()
size = size[size > 5]
latlons = [(index[0], index[1], count) for index, count in size.iteritems()]
latlons = sorted(latlons, reverse=True, key=lambda x: x[2])

count_points = len(latlons)
current_point = 0

def show_current_point(current_point):
    clear_output()
    status_text.value = '{}/{}'.format(current_point + 1, count_points)

    mlat, mlon, _ = latlons[current_point]
    train_mids = set(train_merchants[(train_merchants.lat == mlat) & (train_merchants.lon == mlon)].mid)
    trans = transactions[transactions.mid.apply(lambda x: x in train_mids)]
    
    print mlat, mlon, len(train_mids), trans.shape[0]
    folium_map = folium.Map(location=[mlat, mlon], zoom_start=13)
    folium.Marker([mlat, mlon]).add_to(folium_map)
    folium.features.RectangleMarker(
            bounds=[[mlat - HSIDE, mlon - HSIDE], 
                    [mlat + HSIDE, mlon + HSIDE]],
            color=COLOR_MERCHANT, fill_color=COLOR_MERCHANT, fill_opacity=0.3).add_to(folium_map)
    for _, row in trans.iterrows():
        lat = row.lat
        lon = row.lon
        folium.CircleMarker(location=[lat, lon], radius=8, 
                            color=COLOR_TRANS, fill_color=COLOR_TRANS).add_to(folium_map)

    display(folium_map)
    
def show_next_point(_):
    global current_point
    current_point = (current_point + 1) % count_points
    show_current_point(current_point)
    
def show_prev_point(_):
    global current_point
    current_point = (current_point - 1) % count_points
    show_current_point(current_point)

container = widgets.HBox()
button_next = widgets.Button(description='Next')
button_prev = widgets.Button(description='Prev')
status_text = widgets.Text()
container.children = [button_prev, status_text, button_next]
display(container)
button_next.on_click(show_next_point)
button_prev.on_click(show_prev_point)
show_current_point(current_point)

55.605617 37.486463 18 520


В основном точки транзакций соответствуют точкам мерчантов. Но есть несколько странных мерчантов, например:

1. Пермь, Шоссе Космонавтов - все транзакции раскиданы по России, рядом вообще ничего нет
2. Москва, Марксистская 4 - большинство в Москве, но рядом ничего нет (это та же точка, которую предсказываем для мерчантов со всеми отфильтрованными точками)
3. Линия октябрьской жд - все точки раскинулись вдоль дмитровского шоссе
4. Усть-щербедино - большинство точек в ТРЦ Авиапарк
5. Бизнес-парк станколит - большинство точек в Москве, но без определенных кластеров
6. Красная поляна - большинство точек в Москве, но есть и в Красной поляне, но не рядом
7. Точка не на здании в нижегородском районе - все в Москве, но без кластеров
8. Мега Химки - большинство точек в части здания, но координаты мерчантов отмечены по центру здания
9. Другая точка в Мега Химки, тоже частая, значит, в здании может быть несколько координат мерчантов
    
    !!! если предсказанная точка в Мега Химки, то надо сдвигать ее, чтобы в прямоугольник попали все известные мерчанты !!!

10. Мерчант на мурманском шоссе, хотя большинство транзакций в Мега Дыбенко рядом

    !!! если предсказанная точка попала рядом с Мега Дыбенко, то предсказываем мурманское шоссе !!!
    
11. Мерчант на калужском шоссе, хотя большинство транзакций в Мега Теплый Стан рядом

    !!! если предсказанная точка попала рядом с Мега Теплый Стан, то предсказываем калужское шоссе !!!


В целом есть много ошибок, то есть надо предсказывать не только хорошую точку транзакции, но и попытаться немного сдвинуть предсказание, чтобы попасть в прямоугольник, даже если данные ошибочные.

Для трех зданий выше добавим правила вручную, также попытаемся создать эвристику по сдвигу предсказания, которая улучшает результат.

Посмотрим на мерчанты без номеров домов.

In [13]:
train_merchants = pd.read_csv('data/mtrain.csv')
transactions = pd.read_csv('data/transactions_good.csv')
mids = []
for _, row in train_merchants.iterrows():
    lat = row.lat
    lon = row.lon
    if get_osm_country(lat, lon) == 'РФ' and get_osm_number(lat, lon) == '':
        mids.append((lat, lon, int(row.mid)))

show_merchants(mids, transactions, print_info=True)

1509,55.752485,37.623196
Transactions: 4


Среди точек без номеров зданий есть три основных типа:

1. У здания нет номера - иногда это большие ТЦ, так что здания надо выпарсить вручную из файла.

2. Точки прямо посреди улиц, наверное мерчант не предоставил точный адрес или ошибки ввода. В любом случае надо иметь это в виду, потому что их довольно много. Например Пермь, Шоссе космонавтов - 25 точек.

3. Просто точки без зданий, их довольно мало и с ними ничего нельзя сделать.

### Информация о зданиях

Составим список всех зданий, которые находятся в какой-то окрестности от всех точек транзакций.

Каждый раз парсить файл *.osm.pbf слишком долго, поэтому сохраним все нужные здания в отдельный файл.
Сначала найдем все node ниже 75ой широты, которые находятся не далее 600 метров от какой-то точки транзакции
либо от какой-то точки мерчанта либо входят в здания с тегом shop: mall.

In [6]:
# Честное вычисление расстояния по координатам точек
# http://stackoverflow.com/questions/4913349/haversine-formula-in-python-bearing-and-distance-between-two-gps-points
def haversine(lat1, lon1, lat2, lon2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371000 # Radius of earth in meters
    return c * r

# Честное вычисление работает очень долго, потому что в нем много тригонометрических функций
# Напишем более быструю функцию для нахождения маленьких расстояний (в этом случае поверхность Земли можно
# рассматривать как плоскость)

# длина в метрах одной сотой градуса по широте
LAT_001_DIST = haversine(0, 0, 0.01, 0)
# длина в метрах одной сотой градуса по долготе для каждой широты с шагом 0.01
LON_001_DIST = [haversine(i / 100.0, 0, i / 100.0, 0.01) for i in xrange(7500)]

def haversine_fast_sq(lat1, lon1, lat2, lon2):
    y = (lat1 - lat2) * LAT_001_DIST * 100
    key = int(100.0 * lat1)
    x = (lon1 - lon2) * LON_001_DIST[key] * 100
    return x * x + y * y

def haversine_fast(lat1, lon1, lat2, lon2):
    return math.sqrt(haversine_fast_sq(lat1, lon1, lat2, lon2))

# Переводит latlon координаты в x, y относительно origin
# Предполагается, что latlons находятся в первой четверти относительно origin и расстояния не очень
# большие, то есть поверхность можно рассматривать как плоскость 
def latlon_to_xy(latlons, origin):
    olat, olon = origin
    return [(haversine_fast(lat, lon, lat, olon), haversine_fast(lat, lon, olat, lon)) for lat, lon in latlons]

# Переводит x, y координаты в latlon относительно origin
def xy_to_latlon(xy, origin):
    olat, olon = origin
    lon = olon + xy[0] / (LON_001_DIST[int(100.0 * olat)] * 100)
    lat = olat + xy[1] / (LAT_001_DIST * 100)
    return lat, lon

In [9]:
MAX_CLOSE_DIST = 600
MAX_LAT = 75

# Класс хранит информацию о множестве геокоординат и может по точке быстро отвечать, 
# есть ли какой-то элемент из множества в окрестности точки. Скорость достигается засчет того, что множество
# разбито на прямоугольники размером 0.01x0.01 и проверяются только несколько ближайших прямоугольников.
class LatLonRects:
    
    # расстояние в метрах
    def __init__(self, close_distance):
        self.close_distance = close_distance
        self.rects = defaultdict(list)
        
    def get_rect_id(self, lat, lon):
        return (int(lat * 100), int(lon * 100))
        
    def add(self, lat, lon):
        self.rects[self.get_rect_id(lat, lon)].append((lat, lon))
        
    def has_close(self, lat, lon):
        rlat, rlon = self.get_rect_id(lat, lon)
        lat_add = int(math.ceil(self.close_distance / LAT_001_DIST))
        lon_add = int(math.ceil(self.close_distance / LON_001_DIST[rlat]))
        for i in xrange(rlat - lat_add, rlat + lat_add + 1):
            for j in xrange(rlon - lon_add, rlon + lon_add + 1):
                if (i, j) in self.rects:
                    for plat, plon in self.rects[(i, j)]:
                        if haversine_fast(plat, plon, lat, lon) < self.close_distance:
                            return True
        return False

    
transactions = pd.read_csv('data/transactions_good.csv')
train_merchants = pd.read_csv('data/mtrain.csv')
lat_lon_rects = LatLonRects(MAX_CLOSE_DIST)
# Добавляем все координаты мерчантов и транзакций
for index, _ in transactions.groupby(['lat', 'lon']).size().iteritems():
    lat_lon_rects.add(index[0], index[1])
for index, _ in train_merchants.groupby(['lat', 'lon']).size().iteritems():
    lat_lon_rects.add(index[0], index[1])    


class MallHandler(osmium.SimpleHandler):
    
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.mall_nodes_ids = set()
    
    def area(self, a):
        if 'shop' in a.tags and a.tags['shop'] == 'mall':
            for outer_ring in a.outer_rings():
                for n in outer_ring:
                    self.mall_nodes_ids.add(n.ref)
                    

class CloseNodeHandler(osmium.SimpleHandler):
    
    def __init__(self, lat_lon_rects, mall_nodes_ids, write_file):
        osmium.SimpleHandler.__init__(self)
        self.lat_lon_rects = lat_lon_rects
        self.mall_nodes_ids = mall_nodes_ids
        self.write_file = write_file

    def node(self, n):
        if n.location.lat <= MAX_LAT and (n.id in self.mall_nodes_ids or
                                          self.lat_lon_rects.has_close(n.location.lat, n.location.lon)):
            self.write_file.write('{}\t{}\t{}\n'.format(n.id, n.location.lat, n.location.lon))

            
mall_handler = MallHandler()
mall_handler.apply_file("russia-latest.osm.pbf")
with open('data/close_nodes', 'w') as f:
    node_handler = CloseNodeHandler(lat_lon_rects, mall_handler.mall_nodes_ids, f)
    node_handler.apply_file("russia-latest.osm.pbf")

In [7]:
def clean_tag(s):
    return s.replace('\t', '').replace('\n', '').replace(';', ' ')

In [10]:
class AreaHandler(osmium.SimpleHandler):
    
    def __init__(self, close_nodes, wfile):
        osmium.SimpleHandler.__init__(self)
        self.close_nodes = close_nodes
        self.wfile = wfile
        self.buildings = 0

    def area(self, a):
        for outer_ring in a.outer_rings():
            for n in outer_ring:
                if n.ref not in self.close_nodes:
                    break
            else:
                continue
            break
        else:
            if 'building' in a.tags:
                self.wfile.write(str(a.id))
                self.wfile.write('\t\t')
                self.wfile.write(';'.join(['{};{}'.format(clean_tag(t.k), clean_tag(t.v)) for t in a.tags]))
                for outer_ring in a.outer_rings():
                    self.wfile.write('\t\t')
                    self.wfile.write('\t'.join(['{},{}'.format(*self.close_nodes[n.ref]) for n in outer_ring]))
                self.wfile.write('\n')
                self.buildings += 1


close_nodes = {}
with open('data/close_nodes') as f:
    for line in f:
        nid, lat, lon = line.strip().split('\t')
        close_nodes[int(nid)] = (float(lat), float(lon))

with open('data/close_buildings', 'w') as f:
    way_handler = AreaHandler(close_nodes, f)
    way_handler.apply_file("russia-latest.osm.pbf")

In [8]:
class Shop:
    
    def __init__(self, lat, lon, tags):
        self.lat = lat
        self.lon = lon
        self.tags = tags

        
class Building:
    
    def __init__(self, line):
        tokens = line.strip().split('\t\t')
        self.bid = int(tokens[0])
        tags = tokens[1].split(';')
        self.tags = {tags[2 * i]: tags[2 * i + 1] for i in xrange(len(tags) / 2)}
        self.polygons = []
        for token in tokens[2:]:
            coords = token.split('\t')
            self.polygons.append([[float(t) for t in c.split(',')] for c in coords])
        self.shops = []
        self.centers = []
        self.area = None
        self.bound_rect = None
        
    def add_shop(self, shop):
        self.shops.append(shop)
        
    def add_center(self, lat, lon):
        self.centers.append((lat, lon))
    
    # Выпускаем луч вверх и считаем количество пересеченных сторон
    # если нечетное, то точка внутри
    def is_point_in_polygon(self, lat, lon, polygon):
        count_top = 0
        for i in xrange(len(polygon) - 1):
            slat1 = polygon[i][0]
            slon1 = polygon[i][1]
            slat2 = polygon[i + 1][0]
            slon2 = polygon[i + 1][1]
            if slon1 < lon < slon2 or slon2 < lon < slon1:
                top_lat = slat1 + (slat2 - slat1) * (lon - slon1) / (slon2 - slon1)
                if top_lat > lat:
                    count_top += 1
        return count_top % 2 == 1

    def is_point_in_building(self, lat, lon):
        for polygon in self.polygons:
            if self.is_point_in_polygon(lat, lon, polygon):
                return True
        return False
    
    def dist_to_building(self, lat, lon):
        m = 1000000000
        for polygon in self.polygons:
            for nlat, nlon in polygon:
                h = haversine_fast(lat, lon, nlat, nlon)
                if h < m:
                    m = h
        return m
    
    def get_any_point(self):
        return self.polygons[0][0] if len(self.polygons) > 0 and len(self.polygons[0]) > 0 else None
    
    def draw(self, folium_map, print_tags=True):
        for polygon in self.polygons:
            folium.PolyLine(locations=polygon, color=COLOR_BUILDING, weight=3, opacity=1).add_to(folium_map)
        if print_tags:
            for k, v in self.tags.iteritems():
                print k, v
        for i, shop in enumerate(self.shops):
            if print_tags:
                print str(i) + ':', ';'.join([k + ' ' + v for k, v in shop.tags.iteritems()])
            folium.CircleMarker(location=(shop.lat, shop.lon), radius=10, 
                                color=COLOR_SHOP, fill_color=COLOR_SHOP, fill_opacity=0.3).add_to(folium_map)
            
    def get_centers(self):
        return self.centers
    
    def get_center(self):
        if len(self.centers) == 0:
            print self.bid
            assert False
        if len(self.centers) == 1:
            return self.centers[0]
        res = [0, 0]
        for c in self.centers:
            res[0] += c[0]
            res[1] += c[1]
        return (res[0] / len(self.centers), res[1] / len(self.centers))
    
    def get_area(self):
        return self.area
    
    # top lat, right lon, bottom lat, left lon 
    def get_bound_rect(self):
        if self.bound_rect is None:
            top = bottom = right = left = None
            for a in self.polygons:
                for lat, lon in a:
                    if top is None or lat > top:
                        top = lat
                    if bottom is None or lat < bottom:
                        bottom = lat
                    if right is None or lon > right:
                        right = lon
                    if left is None or lon < left:
                        left = lon
            self.bound_rect = (top, right, bottom, left)
        return self.bound_rect
    
    def is_any_point_in_circle(self, lat, lon, dist_sq):
        for polygon in self.polygons:
            for nlat, nlon in polygon:
                if haversine_fast_sq(lat, lon, nlat, nlon) < dist_sq:
                    return True
        return False
    
    def is_shop_mall(self):
        return 'shop' in self.tags and self.tags['shop'] == 'mall'
    
    def count_shops_of_type(self, shop_type):
        return sum(['shop' in shop.tags and shop.tags['shop'] == shop_type for shop in self.shops])
    
    def count_amenities_of_type(self, amenity_type):
        return sum(['amenity' in shop.tags and shop.tags['amenity'] == amenity_type for shop in self.shops])
    
    def has_shop_with_name(self, name):
        if 'name' in self.tags and self.tags['name'] == name:
            return True
        for shop in self.shops:
            if 'name' in shop.tags and shop.tags['name'] == name:
                return True
        return False
    
    def is_shop_mall_or_supermarket(self):
        return 'shop' in self.tags and (self.tags['shop'] == 'mall' or self.tags['shop'] == 'supermarket')

    def count_good_names(self):
        good_names = {'Макдоналдс', 'Дикси', 'KFC', 'Шоколадница', 'Евросеть', 'Связной', 'Теремок', 'МТС'}
        res = 0
        if 'name' in self.tags and self.tags['name'] in good_names:
            res += 1
        for shop in self.shops:
            if 'name' in shop.tags and shop.tags['name'] in good_names:
                res += 1
        return res
            

class Buildings:
    
    def __init__(self):
        self.rects = defaultdict(list)
        self.building_ids = {}
        with open('data/close_buildings') as f:
            for line in f:
                b = Building(line)
                if len(b.polygons) and b.bid not in self.building_ids > 0:
                    self.add(b)
                    self.building_ids[b.bid] = b
        if os.path.exists('data/close_shops'):
            with open('data/close_shops') as f:
                for line in f:
                    tokens = line.strip().split('\t\t')
                    bid = int(tokens[0])
                    lat, lon = [float(s) for s in tokens[1].split(',')]
                    tags = tokens[2].split(';')
                    tags = {tags[2 * i]: tags[2 * i + 1] for i in xrange(len(tags) / 2)}
                    self.building_ids[bid].add_shop(Shop(lat, lon, tags))
        if os.path.exists('data/buildings_area_center'):
            with open('data/buildings_area_center') as f:
                for line in f:
                    bid, area, centers = line.strip().split('\t')
                    building = self.building_ids[int(bid)]
                    building.area = float(area)
                    for ll in centers.split(';'):
                        lat, lon = ll.split(',')
                        building.add_center(float(lat), float(lon))
    
    def get_by_id(self, bid):
        return self.building_ids[bid]
    
    def get_all_bids(self):
        return self.building_ids.keys()
        
    def add(self, building):
        if not building.get_any_point():
            return
        lat, lon = building.get_any_point()
        rlat = int(lat * 100)
        rlon = int(lon * 100)
        self.rects[(rlat, rlon)].append(building)

    def get_building(self, lat, lon):
        rlat = int(lat * 100)
        rlon = int(lon * 100)
        lat_add = 1
        lon_add = 2
        for i in xrange(rlat - lat_add, rlat + lat_add + 1):
            for j in xrange(rlon - lon_add, rlon + lon_add + 1):
                if (i, j) in self.rects:
                    for building in self.rects[(i, j)]:
                        if building.is_point_in_building(lat, lon):
                            return building
        return None
    
    def get_closest_building(self, lat, lon, max_dist, tag):
        rlat = int(lat * 100)
        rlon = int(lon * 100)
        lat_add = 1
        lon_add = 2
        min_dist = max_dist
        min_building = None
        for i in xrange(rlat - lat_add, rlat + lat_add + 1):
            for j in xrange(rlon - lon_add, rlon + lon_add + 1):
                if (i, j) not in self.rects:
                    continue
                for building in self.rects[(i, j)]:
                    if tag is None or tag in building.tags:
                        h = building.dist_to_building(lat, lon)
                        if h < min_dist:
                            min_dist = h
                            min_building = building
        return min_building
    
    # Возвращает список зданий, любая точка которых на расстоянии <= distance метров от lat, lon
    def get_buildings_in_circle_any(self, lat, lon, distance, distance_mall):
        distance_sq = distance ** 2
        distance_mall_sq = distance_mall ** 2
        rlat = int(lat * 100)
        rlon = int(lon * 100)
        # +300 по долготе потому что здание может быть в одном прямоугольнике по первой точке, но
        # при этом иметь точки в соседних прямоугольниках
        # по широте 1, потому что размер по широте 1100 метров, даже с учетом
        # расстояния 600 метров, надо чтобы здание было 500 метров в длину, таких зданий очень мало
        lat_add = 1
        lon_add = int(math.ceil((distance + 300) / LON_001_DIST[rlat]))
        buildings = []
        for i in xrange(rlat - lat_add, rlat + lat_add + 1):
            for j in xrange(rlon - lon_add, rlon + lon_add + 1):
                if (i, j) in self.rects:
                    for building in self.rects[(i, j)]:
                        if not building.is_shop_mall():
                            if building.is_any_point_in_circle(lat, lon, distance_sq):
                                buildings.append(building)
        lat_add = 1
        lon_add = int(math.ceil((distance_mall + 300) / LON_001_DIST[rlat])) + 1
        for i in xrange(rlat - lat_add, rlat + lat_add + 1):
            for j in xrange(rlon - lon_add, rlon + lon_add + 1):
                if (i, j) in self.rects:
                    for building in self.rects[(i, j)]:
                        if building.is_shop_mall():
                            if building.is_any_point_in_circle(lat, lon, distance_mall_sq):
                                buildings.append(building)
        return buildings
    
    # Возвращает список зданий, центр которых на расстоянии <= distance метров от lat, lon
    def get_buildings_in_circle_center(self, lat, lon, distance, distance_mall):
        rlat = int(lat * 100)
        rlon = int(lon * 100)
        lat_add = 1
        lon_add = int(math.ceil((distance + 300) / LON_001_DIST[rlat]))
        buildings = []
        for i in xrange(rlat - lat_add, rlat + lat_add + 1):
            for j in xrange(rlon - lon_add, rlon + lon_add + 1):
                if (i, j) in self.rects:
                    for building in self.rects[(i, j)]:
                        if not ('shop' in building.tags and building.tags['shop'] == 'mall'):
                            center = building.get_center()
                            if haversine_fast(center[0], center[1], lat, lon) < distance:
                                buildings.append(building)
        lat_add = 1
        lon_add = int(math.ceil((distance_mall + 300) / LON_001_DIST[rlat])) + 1
        for i in xrange(rlat - lat_add, rlat + lat_add + 1):
            for j in xrange(rlon - lon_add, rlon + lon_add + 1):
                if (i, j) in self.rects:
                    for building in self.rects[(i, j)]:
                        if 'shop' in building.tags and building.tags['shop'] == 'mall':
                            center = building.get_center()
                            if haversine_fast(center[0], center[1], lat, lon) < distance_mall:
                                buildings.append(building)
        return buildings

buildings = None
def load_buildings():
    global buildings
    if buildings is None:
        buildings = Buildings()

Также сохраним в отдельный файл данные по магазинам, которые находятся в этих зданиях.

In [15]:
class ShopHandler(osmium.SimpleHandler):
    
    def __init__(self, close_nodes, buildings, wfile):
        osmium.SimpleHandler.__init__(self)
        self.close_nodes = close_nodes
        self.buildings = buildings
        self.wfile = wfile
        self.save_for_tags = [('amenity', {'cafe', 'fast_food', 'cinema', 
                                           'restaurant', 'pharmacy', 'bar', 'clinic',
                                           'pub', 'food_court', 'dentist',
                                           'hospital', 'doctors', 'car_wash', 'biergarten'}),
                              ('shop', '*'), ]
        self.save_tags = {'amenity', 'shop', 'opening_hours', 'name', 'room'}
        
    def write_to_file(self, bid, latlon, tags):
        self.wfile.write(str(bid))
        self.wfile.write('\t\t')
        self.wfile.write('{},{}'.format(*latlon))
        self.wfile.write('\t\t')
        tags = [clean_tag(t.k) + ';' + clean_tag(t.v) for t in tags if t.k in self.save_tags]
        self.wfile.write(';'.join(tags))
        self.wfile.write('\n')

    def node(self, n):
        for name, value in self.save_for_tags:
            if name in n.tags and (value == '*' or n.tags[name] in value):
                building = self.buildings.get_building(n.location.lat, n.location.lon)
                if building is not None:
                    self.write_to_file(building.bid, (n.location.lat, n.location.lon), n.tags)
                break
                
    def way(self, w):
        # Для внутренних магазинов сохраняем одну точку и id всего здания
        if 'room' in w.tags and w.tags['room'] == 'store':
            for n in w.nodes:
                nr = n.ref
                if nr in self.close_nodes:
                    building = self.buildings.get_building(*self.close_nodes[nr])
                    if building is not None:
                        self.write_to_file(building.bid, self.close_nodes[nr], w.tags)
                    break

close_nodes = {}
with open('data/close_nodes') as f:
    for line in f:
        nid, lat, lon = line.strip().split('\t')
        close_nodes[int(nid)] = (float(lat), float(lon))
    
load_buildings()
with open('data/close_shops', 'w') as f:
    shop_handler = ShopHandler(close_nodes, buildings, f)
    shop_handler.apply_file("russia-latest.osm.pbf")

Закэшируем в файл площадь и центр каждого здания.

In [9]:
def area_for_polygon(polygon):
    result = 0
    for i in range(0, len(polygon) - 1):
        result += (polygon[i][0] * polygon[i+1][1]) - (polygon[i+1][0] * polygon[i][1])
    return result / 2.

def centroid_for_polygon(polygon, S):
    result_x = 0
    result_y = 0
    for i in range(0, len(polygon) - 1):
        result_x += (polygon[i][0] + polygon[i+1][0]) * ((polygon[i][0] * polygon[i+1][1]) - (polygon[i+1][0] * polygon[i][1]))
        result_y += (polygon[i][1] + polygon[i+1][1]) * ((polygon[i][0] * polygon[i+1][1]) - (polygon[i+1][0] * polygon[i][1]))
    result_x /= (S * 6.0)
    result_y /= (S * 6.0)

    return (result_x, result_y)

load_buildings()
bids = buildings.get_all_bids()
with open('data/buildings_area_center', 'w') as f:
    for bid in tqdm(bids):
        building = buildings.get_by_id(bid)
        rect = building.get_bound_rect()
        origin = (rect[2], rect[3])
        all_s = 0
        all_c = []
        for polygon in building.polygons:
            xy = latlon_to_xy(polygon, origin)
            s = area_for_polygon(xy)
            all_s += s
            all_c.append(xy_to_latlon(centroid_for_polygon(xy, s), origin))
        f.write('{}\t{:.4f}\t{}\n'.format(bid, all_s, ';'.join(['{:.6f},{:.6f}'.format(*c) for c in all_c])))

100%|██████████| 1631802/1631802 [00:43<00:00, 37238.09it/s]


Посмотрим на все мерчанты подробно (здание, все магазины в здании, все транзакции)

In [9]:
load_buildings()

transactions = pd.read_csv('data/transactions_good.csv')
train_merchants = pd.read_csv('data/mtrain.csv')
current_merchant = 0
count_merchants = train_merchants.shape[0]

def show_merchant(num_merchant):
    clear_output()
    status_text.value = '{}/{}'.format(num_merchant + 1, count_merchants)
    
    row = train_merchants.iloc[num_merchant, :]
    trans = transactions[transactions.mid == row.mid]
    lat, lon = row.lat, row.lon
    folium_map = folium.Map(location=(lat, lon), zoom_start=18)
    building = buildings.get_building(lat, lon)
    building.draw(folium_map)
    folium.CircleMarker(location=(lat, lon), radius=4, 
                  color=COLOR_MERCHANT, fill_color=COLOR_MERCHANT).add_to(folium_map)
    for _, row in trans.iterrows():
        folium.CircleMarker(location=(row.lat, row.lon), radius=5, 
                  color=COLOR_TRANS, fill_color=COLOR_TRANS).add_to(folium_map)
    
    print 'TRANSACTIONS: ' + str(trans.shape[0])

    display(folium_map)
    
def show_next_merchant(_):
    global current_merchant
    current_merchant = (current_merchant + 1) % count_merchants
    show_merchant(current_merchant)
    
def show_prev_merchant(_):
    global current_merchant
    current_merchant = (current_merchant - 1) % count_merchants
    show_merchant(current_merchant)

container = widgets.HBox()
button_next = widgets.Button(description='Next')
button_prev = widgets.Button(description='Prev')
status_text = widgets.Text()
container.children = [button_prev, status_text, button_next]
display(container)
button_next.on_click(show_next_merchant)
button_prev.on_click(show_prev_merchant)
show_merchant(current_merchant)

addr:housenumber 1
building retail
name Гиппо
shop mall
building:levels 2
addr:street Нагорная улица
0: shop furniture;name ДОРМастерс
1: opening_hours 24/7;amenity pharmacy;name Вита Норд
2: shop laundry;opening_hours Mo-Fr 11:00-20:00  Sa-Su 11:00-19:00;name Лавандерия
3: shop clothes;opening_hours Mo-Fr 11:30-20:00  Sa-Su 11:30-19:00;name La-Belle
4: shop clothes;opening_hours Mo-Fr 11:00-21:00  Sa-Su 11:00-20:00;name Джулия
5: shop toys;opening_hours 11:00-20:00;name Конфетти
6: shop pet;opening_hours Mo-Fr 11:00-21:00  Sa-Su 11:00-20:00;name Лап-Усик
7: shop gift;opening_hours 11:00-20:00;name Оранжевое настроение
8: shop chemist;opening_hours Mo-Fr 11:00-21:00  Sa-Su 11:00-20:00;name ШиК
9: shop clothes;opening_hours Mo-Fr 11:00-20:00  Sa-Su 11:00-19:00;name Я мама
10: shop supermarket;opening_hours 24/7;name Петровский
TRANSACTIONS: 31


Построим распределение количества мерчантов в здании и распределение количества различных точек мерчантов в здании.

In [10]:
load_buildings()
train_merchants = pd.read_csv('data/mtrain.csv')
count = defaultdict(int)
for _, row in train_merchants.iterrows():
    building = buildings.get_building(row.lat, row.lon)
    if building:
        count[building.bid] += 1
print Counter(count.values())

Counter({1: 2859, 2: 518, 3: 115, 4: 69, 6: 23, 5: 19, 7: 12, 8: 8, 9: 5, 13: 4, 11: 3, 12: 3, 10: 2, 14: 2, 17: 1, 18: 1, 19: 1, 20: 1, 23: 1, 28: 1, 29: 1, 34: 1, 37: 1, 39: 1})


In [12]:
load_buildings()
train_merchants = pd.read_csv('data/mtrain.csv')
count = defaultdict(set)
for _, row in train_merchants.iterrows():
    building = buildings.get_building(row.lat, row.lon)
    if building:
        count[building.bid].add((row.lat, row.lon))
print Counter([len(v) for v in count.values()])

Counter({1: 3561, 2: 78, 3: 12, 4: 1})


Видим, что в основном мерчанты повторяются в зданиях в одних и тех же точках. Но есть несколько зданий, где мерчанты в разных точках.

Посмотрим на самые встречающиеся названия торговых точек.

In [12]:
load_buildings()
train_merchants = pd.read_csv('data/mtrain.csv')
names = defaultdict(int)
bids = set()
for _, row in train_merchants.iterrows():
    building = buildings.get_building(row.lat, row.lon)
    if building and building.bid not in bids:
        bids.add(building.bid)
        if 'name' in building.tags:
            names[building.tags['name']] += 1
        for shop in building.shops:
            if 'name' in shop.tags:
                names[shop.tags['name']] += 1
names = [(k, v) for k, v in names.iteritems()]
names = sorted(names, key=lambda x: x[1], reverse=True)[:50]
for k, v in names:
    print k, v

Макдоналдс 200
Дикси 170
KFC 114
Шоколадница 87
Евросеть 82
Связной 81
Продукты 78
Теремок 70
МТС 64
Перекресток 62
Subway 55
Спортмастер 52
Горздрав 50
Бургер Кинг 48
Л'Этуаль 45
М.Видео 40
Эльдорадо 40
Кофе Хауз 40
Билайн 39
Пятёрочка 38
36,6 34
Мегафон 34
Якитория 33
Детский мир 32
Burger King 31
Лента 31
МегаФон 31
Магнолия 30
Ашан 29
H&M 29
Ригла 28
Верный 27
Виктория 27
Adidas 24
Магнит 24
Перекрёсток 24
Media Markt 24
Крошка Картошка 23
Бетховен 21
Цветы 21
Даблби 21
Первая полоса 21
DNS 21
Старбакс 20
Prime 20
Ароматный мир 20
Tele2 20
Евразия 20
Сбарро 19
Billa 19


### Общий ход решения

1. Будем рассматривать транзакции как объекты, для каждой транзакции сгенерируем признаки.
2. В качестве таргета возьмем бинарный признак, входит ли транзакция в окрестность мерчанта (расстояние по широте и долготе меньше 0.002).
3. Обучимся на трейне и предскажем вероятность на тесте.
4. Возьмем транзакцию с максимальной вероятностью и предскажем точку мерчанта максимально близко к этой транзакции, но чтобы прямоугольник захватывал известные точки мерчантов поблизости.

Признаки и таргет генерируются в файлах features_*.py, обучение и смещение в learn.py.